<a href="https://colab.research.google.com/github/maktaurus/ML-Work/blob/main/ONNX_Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conversion of TF model to ONNX and Quantization

Checkout below documentation for further understanding

https://onnxruntime.ai/docs/tutorials/tf-get-started.html

https://github.com/onnx/tensorflow-onnx

https://onnxruntime.ai/docs/performance/model-optimizations/quantization.html#:~:text=Quantization%20in%20ONNX%20Runtime%20refers,quantization%20of%20an%20ONNX%20model.&text=zero_point%20represents%20zero%20in%20the,is%20used%20in%20many%20CNNs.


In [ ]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.utils import load_img, img_to_array, array_to_img
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras import layers
from tensorflow.keras.metrics import Accuracy,TruePositives,TrueNegatives,FalsePositives,FalseNegatives,Recall,Precision,AUC
import zipfile
import tensorflow_hub as hub

from tensorflow.keras import layers

In [ ]:
!pip install -U tf2onnx
!pip install onnxruntime

In [ ]:
import tf2onnx
import onnxruntime as rt

Provide the specification for input. The tf2onnx will give two output.
Directly pass the raw pretained model and provide the output path to save the .onnx format file.

In [ ]:
spec = (tf.TensorSpec((None,224,224,3),tf.float32,name="input"),)
model_proto, modl = tf2onnx.convert.from_keras(vit_model,
                                               input_signature=spec,
                                               opset=13,
                                               output_path="vit.onnx")
output_names = [n.name for n in model_proto.graph.output]

In [ ]:
img = cv.imread("/content/test/d12.jpg")
img = cv.resize(img,(224,224))
img = img.astype(np.float32)
img = np.expand_dims(img,axis=0)

In [ ]:
providers = ['CPUExecutionProvider']

m = rt.InferenceSession("vit.onnx", providers=providers)
onnx_pred = m.run(output_names, {"input": img})
onnx_pred

In [ ]:
class_names[np.argmax(onnx_pred)]

# Quantization with onnx

In [ ]:
from onnxruntime.quantization import quantize_dynamic,QuantType

In [ ]:
model_f32 = "/content/vit.onnx"
model_quant = "/content/vit_quantized.onnx"

quantized_model = quantize_dynamic(model_f32,model_quant,weight_type=QuantType.QUInt8)

In [ ]:
providers = ['CPUExecutionProvider']

m = rt.InferenceSession("vit_quantized.onnx", providers=providers)
onnx_pred = m.run(output_names, {"input": img})
onnx_pred

In [ ]:
class_names[np.argmax(onnx_pred)]